In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import warnings
import datetime
import math
import json

import pandas as pd
import geopandas as gpd
import numpy as np 
from tqdm.auto import tqdm

import src.data_utils as u

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 100)

## Loading data

In [2]:
# If some files don't exist you should generate them via `notebooks/additional_data_preprocessing.ipynb`.

road_segments_path = '../data/road_segments_processed/road_segments_vds_uber.shp'
sample_submission_path = '../data/SampleSubmission.csv'
train_cleaned_path = '../data/train_cleaned.csv'
result_weather_path = '../data/weather_processed/weather.csv'
cameras_count_path = '../data/SANRAL_processed/cameras_counts.csv'
vds_locations_path = '../data/SANRAL_processed/vds_locations.csv'
result_hourly_path = '../data/SANRAL_processed/VDS_hourly_all.csv'
cameras_comm_dates_path = '../data/SANRAL_processed/cameras_comnissionning_dates.csv'
result_segment_ttime_path = '../data/uber_processed/segment_ttime_daily.csv'
sid_neigh_path = '../data/uber_processed/sid_neighbors.json'

# Paths for saving resulted files:
train_pkl_path = '../data/train.pkl'
test_pkl_path = '../data/test.pkl'

In [3]:
road_segments        = gpd.read_file(road_segments_path)

sid_neighbors        = json.load(open(sid_neigh_path, 'r'))

sample_sub           = pd.read_csv(sample_submission_path)
train_raw            = pd.read_csv(train_cleaned_path, parse_dates=['Occurrence Local Date Time'])
weather              = pd.read_csv(result_weather_path, parse_dates=['datetime'])
road_segment_cam_cnt = pd.read_csv(cameras_count_path, sep=';')
VDS_locations        = pd.read_csv(vds_locations_path) 
VDS_hourly           = pd.read_csv(result_hourly_path, parse_dates=['Date of Collection Period'])
commission_dates     = pd.read_csv(cameras_comm_dates_path)
segment_ttime_daily  = pd.read_csv(result_segment_ttime_path)

In [4]:
sample_sub[['datetime','segment_id']] = sample_sub['datetime x segment_id'].str.split(' x ',expand=True)
test = sample_sub[['datetime x segment_id', 'datetime', 'segment_id', 'prediction']]
test = test.rename(columns={'prediction': 'y'})

## Let's start feature engineering. . .

In [5]:
train_raw['longitude'] = pd.to_numeric(train_raw['longitude'], errors='coerce')
train_raw.dropna(subset=['longitude'], inplace=True)
train_raw['longitude'] = train_raw['longitude'].astype('float')
train_raw.head(1)


,EventId,Occurrence Local Date Time,Reporting Agency,Cause,Subcause,Status,longitude,latitude,road_segment_id
0,60558,2016-01-01 00:53:00,Cam,Stationary Vehicle,Vehicle On Shoulder,Closed,18.540896,-33.888275,S0B3CGQ


In [6]:
sids = train_raw['road_segment_id'].unique()
tr   = pd.DataFrame({'datetime': pd.date_range('2016-01-01', '2019-01-01 00:00:00', freq="1h")})

for sid in sids:
    tr[str(sid)] = 0
    events = train_raw.loc[train_raw['road_segment_id'] == sid]
    dts = events['Occurrence Local Date Time'].dt.floor('H')
    dates = dts.astype(str).unique()
    tr.loc[tr['datetime'].isin(dates), sid] = 1
tr.head(1)

,datetime,S0B3CGQ,RYJYAPI,U3KP57C,RY0TRQ8,8LOVJZ3,X4UA382,0QR8FDW,DZABHQW,EKZN1VM,H9XYX9Q,03RHJ3G,FTX7LB8,Q03FQ74,5AC6YRF,1RWEBRU,DCB3LQM,HN83F3L,SPPGKO2,C1ERJ22,AJRKP0C,UUZT4OE,WAC029P,JT4HGZ2,8Z5P9T1,UAXX1ZE,74A6FL0,2M1UBKJ,E5D3KEB,NQSL7FX,S200A1J,ETFFDX5,R6JUSZT,EDXK0HP,N5NO82G,5DHNI4S,VUYACXV,VMCY8BI,927BHUF,3MW0MN9,T182CUC,F53D1DI,7QBQK9L,MEOYV3B,UB5HE92,MJ9YER2,HIUQ2RB,DRNRL0M,LO3764F,OBXISZN,P36URXV,W70WUF7,77YRHPV,MM6HDEQ,LXZFSP3,D7SS5LM,YM2J7ZE,812PNMZ,M3INBBC,93GA3XB,OVJNYB2,VHGABXK,YLQRLAD,C6JTJJC,93O3U6U,GO93XEE,SJ29DQW,K48Y5T9,87Z5O7Q,2YQQ9WS,IEBUIXM,YGRV6SD,DUKT4RG,CZBWW0Z,S9PC5LR,VBUCV9N,T64WM0N,1DDYC6F,Q0VL8BD,WG6X4Y0,MX8CVMX,6BYSPD6,SNTDQP9,O271SQN,H8PYLH4,727J2TV,LO1OZJ0,MC84I0T,XHKKOUG,P0XX6RS,6OTRJQF,A0RUUQL,4K39AFD,OD60AK9,KL4UM68,D1U6OOF,QDRC5UW,7AKUBY4,CJ0BLSW,S7S8GMD,HM3QRDM,8FBHDH2,GNGA5AC,UY59I9M,MRQ81XJ,AGD8UQ2,XO4JYNU,YQCBO2Y,7HICT0R,LNO3W8J,8ITYQ2Z,5OR8UGL,0QQIE2L,I25JSGF,GDVJYQW,DRI712Q,GBN0X51,7PRTR79,7EHES8A,WE8VOFH,IV5F6YM,FSOOCGH,SE831U7,IK67XHB,XNUEEZN,JW0WKQ9,YEDB2ES,0OH5J23,XINDPM1,J0SM52K,VODEAWO,MA3ZWVH,ON7Z6I0,AEDN7D7,QORS7OP,3YS1KQ9,HQ54B8O,A3B2HGD,2NHF7YC,QFRMXC4,M4U0X5G,7YTZ0EM,UCJGSIW,HNEAFHF,WVXBO5X,IXSLKS7,PLZ8PGL,YQRHLO2,PCIQQ7E,A31P11M,MP664QK,EZG7A38,2FN9T8O,N1MTRIJ,WS0J1CU,3GCD5AD,3ZQNS8O,GFX7S4B,H48JVZQ,BDDVLQE,YMQX3CJ,V19MLWF,ZL4WLO2,6LJRHZ3,0W39BFY,XOCWI97,5XWBAKZ,WJBR1L2,Z13SS1G,LG58LKX,CTB99FS,W0EUG1C,RZK0UM0,XS7M5WZ,Z2OSI9E,0TMS99B,XKFABCI,B6D0HHC,VWOSZKP,S2QPOTD,IQS1GLY,4SJWPE0,L4CWZBU,UXERJVK,H9QJECU,1K4ZYII,16WNX7T,BC5XKSB,0GZ5KS3,3IQ1GWG,V1BNX3D,8PA0RQ9,3QSXJH3,3VORNT0,GRI93Z2,5093ZHY,L2X64FB,68KOZH6,BK0B4IL,X9J910I,0TAWLCK,OFAXGKF,2BTEHF3,WRJK3P3,4K0TGLP,WUGV4DU,MPGNI7M,WO118LR,ZT6MOAL,3WK9HI2,8FIZY1M,FHYWZ07,VI1GKEJ,BL0PD9J,SZJISW0,IUTMY1U,WWFC10X,9PORL0H,VTJ1O97,C3K236I,BG1AFJU,LK5JIX6,NRWU3QW,1BJRIL1,F8OXXG7,74CTTH5,6CP0S10,G98H7FR,1Q44FGG,EK3DHQ6,T1DB85Y,WL17NEO,396D4HI,0F3OY57,9HDQLZ0,I8VWS8W,N9A4I1K,NCMIGB7,GU8ECHB,SQ0TZ3H,4HHV563,M2KOUEK,P9PBFAJ,ASFER6E,1JRNIP6,DRFEFT4,OCR113O,1S5XWTK,5PYA0N2,FKBUC13,M223F8G,UEP9HM4,V59S0GZ,TK9DSXW,4ZR4GL4,IQEYE3S,Z4HC8K2,K27GB0B,MTPU2BE,ZCX0S5O,QM4QKAZ,OCFS8ZD,YF1XDKK,F5UCVMI,GUA17R4,3NAG1R5,9EXF4O5,50TVADD,0ICKV72,1G21L7Q,JRZWRIW,Q7OLWGV,HL23AIZ,Z4VBV3L,SUP29NG,TC7A716,6A90XDM,6EAJWW6,S4MOE5G,MRHTO2C,B9V4IAJ,HH4TZKC,4A66E8Q,P7RBBFB,Y5Z7TXB,H8R7OJ2,8WHR0ZV,8PK91S2,XTC73NQ,Z2I5UQC,OWCF2MH,FWO1XRQ,N90YL69,JY8VL6C,YAPO4I4,VVJ158O,REN8XE5,1VFLTND,5QTO800,0B0QOEN,G181X8F,VO6FQAZ,PD4MTSH,QVZKPQD,SJUWTEK,47SP0JH,DMGFH2Z,TD8YB0R,2B7HTHS,1BB1WRK,PRRLPFL,3BMH2Q5,9484GS3,5OV4U4C,WHW1BZK,M43AKN2,DVQD3DE,F1LYCFO,75BPNAM,96FMXIF,JUJL6W1,GX2UELI,CCMG98M,HB933EE,K2E55RT,89VZ0E3,YU41XSB,0ROEFOK,792705Z,IMJY7S7,XW393UI,L6EU3EB,1QSGDUE,DD8YXZF,RWSJ6AY,F14EJLW,TBYQ54D,S55R1TY,GPYNUA8,2XKQDM5,HSJ0WM7,AQW5HO1,TKB7K8T,71YNY81,70OVULT,IY03OM4,UABQ9EK,TBPN1BF,WTTU2KQ,JCFWX6E,MORJH71,WELMR24,2Q8T4YB,GH5QX19,2M4LCU0,LLGN7DY,VPNLAYY,LV4XGT5,WLFBY9T,JJGXEJ4,D5DGOZ9,V5PX6ZH,G5753GU,BN3EJU4,VXS8NZ1,PDDTLDP,FVZ3HQS,5Y5V87D,AEY24LO,WOSAB94,RO046L9,5HTE1IC,LUOEP7G,XTTWKZM,I2POO7A,7O594ZJ,87QMHBI,Q2V2552,QUHOO8R,2MHBCT2,XTZ54JY,WPD6G14,SQ3W7J8,V4ITGH1,9NXZ103,3L7MZUR,RB56HQI,24P30PE,G2LL3S8,044ZYVD,JNEJVKK,CRAU339,VU4GFO5,8SB405D,2J6C2D5,PRUSMVO,ZFMKTPH,TONSERE,3A5K1RU,I6G60AH,6SV0XZ0,GWTSM5V,EIIZE13,ULMQMUT,JZGFZ7Z,J151TEQ,6Q68JYJ,LFUJ30W,SSS7BK6,EKVTVMV,NZUF7M4,B3XNGYS,31F34EZ,KFYGRO8,AAS3KXB,XZZ3D4U,8LSZDSC,OHQYM1R,YEWTYV9,LUOD99L,Q9S9V3Q,T7XSLG6,49ZDYOH,KVQ5DMG,PV3AVCT,AZ1BUKH,086LLYS,C0RCCDP,GZ48XRL,IKN5JDM,9WPSDH2,NG4X2MD,Q17B8NG,UHFPI9X,PXI00H9,JKTV1QW,V5A8ZZZ,FUPJ3Q2,YNCIDMW,6VRO5T2,QXDLKB7,P5N4Z0R,J7MH0RF,Y4B7JOT,ZQJNJPJ,TRTR582,T14JPJU,MQE3R88,IH75D0V,327YML6,NFEBWR6,KNG6Z86,B1VWF3B,9FQFQ91,5VK3U7Q,C6FASR5,YGKOTWP,8MX4B9C,XPEXW89,J2L78G3,6K0F6S9,5QOCGMJ,EPX268G,6BNXT5I,0PU7VDI,EVFKBUY,9DFWVJX,EVANFZB,EJUD4D6,AUFDV1E,G5UHNNF,MFUUB92,KDFE19M,H7OMXI9,X17ILML,C62YWEF,EUXS47I,CK5WH39,GMEHC97,YSM1T15,8EI7FOX,JTY12HG,HFQZU6J,NL4FSPV,G850GMA,92BILMX,Q54XDYI,XBYNXP7,LAZLJ7E,GLFV0XV,8VPPU5N,KL3QK6O,9VXCPYN,4KQ8VO6,9LMPLRW,6E6M9NN,BUJJNEA,Q75ZW1Q,XFMDDJ7,ICNJ23

In [7]:
train = pd.DataFrame({
    'datetime x segment_id': np.concatenate([[str(x) + " x " + str(c) 
                                             for c in sids] 
                                             for x in tr['datetime']]),
    'datetime': np.concatenate([[str(x) for c in sids] for x in tr['datetime']]),
    'segment_id': np.concatenate([[str(c) for c in sids] for x in tr['datetime']]),
    'y': tr[sids].values.flatten()
})
train = train.loc[train['segment_id'].isin(test['segment_id'].tolist())]
train.head(1)

,datetime x segment_id,datetime,segment_id,y
0,2016-01-01 00:00:00 x S0B3CGQ,2016-01-01 00:00:00,S0B3CGQ,1


## Road Segments

In [8]:
road_segments['LANES'].replace(0,1,inplace=True)
road_segments['lane_width']     = road_segments['WIDTH'] / road_segments['LANES']
road_segments['sinuosity']      = road_segments.geometry.apply(u.get_sinuosity) - 1
road_segments['lon_centroid']   = road_segments.geometry.apply(lambda s: s.centroid.coords[0][0])
road_segments['lat_centroid']   = road_segments.geometry.apply(lambda s: s.centroid.coords[0][1])
road_segments['dist_to_center'] = np.sqrt((road_segments.lon_centroid - 18.421148) ** 2 +
                                          (road_segments.lon_centroid + 33.920460) ** 2)  # coords of CT center
road_segments['orientation']  = road_segments['geometry'].apply(u.get_orientation)
road_segments.drop(['REGION','geometry'],axis=1,inplace=True)

#Adding camera count on each segment
road_segments = road_segments.merge(road_segment_cam_cnt, left_on='segment_id', right_on='segment_id', how='left')
road_segments[['camera_count','vms_count','vds_count']] = road_segments[['camera_count','vms_count','vds_count']].fillna(0)
road_segments['vds_id'] = road_segments['vds_id'].replace({'VDS139IO':'VDS139', 
                                                           'VDS912IO':'VDS912',
                                                           'VDS913IO':'VDS913',
                                                           'VDS914IO':'VDS914'}, regex=True)
road_segments.head(1)

,ROADNO,CLASS,WIDTH,LANES,SURFTYPE,PAVETYPE,CONDITION,length_1,segment_id,vds_id,num,main_route,lane_width,sinuosity,lon_centroid,lat_centroid,dist_to_center,orientation,camera_count,vms_count,vds_count
0,R300,Primary,20.2,2,Paved,FLEX,Good,471.207,D1U6OOF,VDS306,0,1.0,10.1,0.000064,18.653044,-33.975166,52.574016,11.516334,0.0,0.0,0.0


In [9]:
road_segments['num'] = road_segments['num'].astype(str)
road_segments['neighbors_to_center']   = road_segments['num'].astype(str)
road_segments['neighbors_from_center'] = road_segments['num'].astype(str)

for direction in sid_neighbors:
    for sid, neighbor in sid_neighbors[direction].items():
        neighbor = ','.join(str(x) for x in neighbor)
        road_segments['neighbors_{0}'.format(direction)].replace(sid, neighbor, inplace=True)
        
road_segments['neighbors_to_center'][road_segments['neighbors_to_center'] == road_segments['num']] = 'None'
road_segments['neighbors_from_center'][road_segments['neighbors_from_center'] == road_segments['num']] = 'None'

road_segments['neighbors'] = road_segments['neighbors_to_center'] + ',' + road_segments['neighbors_from_center']

is_not_none = lambda x: x != 'None'
road_segments['neighbors'] = road_segments['neighbors'].apply(lambda s: 
                                                              set(filter(is_not_none, list(str(s).split(','))))
                                                             )
num_to_neighbors = dict(zip(road_segments.num, road_segments['neighbors']))

road_segments.drop(['neighbors', 'neighbors_to_center', 'neighbors_from_center'], axis=1, inplace=True)

road_segments.head(1)

,ROADNO,CLASS,WIDTH,LANES,SURFTYPE,PAVETYPE,CONDITION,length_1,segment_id,vds_id,num,main_route,lane_width,sinuosity,lon_centroid,lat_centroid,dist_to_center,orientation,camera_count,vms_count,vds_count
0,R300,Primary,20.2,2,Paved,FLEX,Good,471.207,D1U6OOF,VDS306,0,1.0,10.1,0.000064,18.653044,-33.975166,52.574016,11.516334,0.0,0.0,0.0


## VDS_data
 

In [10]:
VDS_hourly = u.clean_vds_data(vds_hourly=VDS_hourly, vds_locations=VDS_locations)

tr = pd.DataFrame({'datetime': pd.date_range('2016-01-01', '2019-03-31 23:00:00', freq="1h")})

VDS_list = VDS_hourly.vds_id.unique()
VDS_hours = pd.DataFrame({
    'datetime': np.concatenate([[str(x) for c in VDS_list] for x in tr['datetime']]),
    'vds_id': np.concatenate([[str(c) for c in VDS_list] for x in tr['datetime']])}
)

VDS_hours['datetime'] = VDS_hours['datetime'].astype('datetime64[ns]')
VDS_hourly = VDS_hours.merge(VDS_hourly, how='left', on=['datetime', 'vds_id'])

In [11]:
VDS_hourly['hour']    = VDS_hourly.datetime.dt.hour
VDS_hourly['weekday'] = VDS_hourly.datetime.dt.weekday
VDS_hourly['year']    = VDS_hourly.datetime.dt.year
VDS_hourly['month']   = VDS_hourly.datetime.dt.month
VDS_hourly['quarter'] = (VDS_hourly.datetime.dt.month - 1) // 3 + 1

VDS_hourly['traffic_unknown'] = VDS_hourly['traffic_total'].isnull().astype(int)
VDS_hourly['speed_unknown']   = VDS_hourly['avg_speed'].isnull().astype(int)

In [12]:
cols_fill_na = ['traffic1', 'traffic2', 'traffic3', 'traffic_total', 'avg_speed']
for f in cols_fill_na:
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id', 'year', 'month', 'weekday', 'hour'])[f].transform('mean'))
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id', 'month', 'weekday', 'hour'])[f].transform('mean'))
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id', 'weekday', 'hour'])[f].transform('mean'))
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id', 'hour'])[f].transform('mean'))
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id', 'weekday'])[f].transform('mean'))
    VDS_hourly[f] = VDS_hourly[f].fillna(VDS_hourly.groupby(['vds_id'])[f].transform('mean'))

In [13]:
VDS_hourly['mean_avg_speed'] = VDS_hourly.groupby(['vds_id', 'weekday', 'hour'])['avg_speed'].transform('mean')
VDS_hourly['std_avg_speed']  = VDS_hourly.groupby(['vds_id', 'weekday', 'hour'])['avg_speed'].transform('std')

VDS_hourly['mean_traffic']   = VDS_hourly.groupby(['vds_id', 'weekday', 'hour'])['traffic_total'].transform('mean')
VDS_hourly['std_traffic']    = VDS_hourly.groupby(['vds_id', 'weekday', 'hour'])['traffic_total'].transform('std')

VDS_hourly['rel_diff_avg_speed'] = (VDS_hourly['avg_speed'] - VDS_hourly['mean_avg_speed']) / VDS_hourly['mean_avg_speed']
VDS_hourly['rel_diff_traffic']   = (VDS_hourly['traffic_total'] - VDS_hourly['mean_traffic']) / VDS_hourly['mean_traffic']

In [14]:
VDS_hourly = u.add_lag_features_vds(VDS_hourly)
VDS_hourly = VDS_hourly.drop(['hour', 'weekday', 'year', 'month', 'quarter'], axis=1)

VDS_hourly.head(1)

,datetime,vds_id,traffic1,traffic2,traffic3,traffic_total,avg_speed,traffic_unknown,speed_unknown,mean_avg_speed,std_avg_speed,mean_traffic,std_traffic,rel_diff_avg_speed,rel_diff_traffic,delta_avg_speed_last_hour,delta_avg_speed_next_hour,delta_traffic_total_last_hour,delta_traffic_total_next_hour,delta_rel_diff_avg_speed_last_hour,delta_rel_diff_avg_speed_next_hour,delta_rel_diff_traffic_last_hour,delta_rel_diff_traffic_next_hour
0,2016-01-01,VDS101,391.791667,7.333333,10.083333,409.208333,104.175833,1,1,103.216256,2.410992,389.450948,144.916352,0.009297,0.050731,NaN,-2.3525,NaN,-178.041667,NaN,-0.007323,NaN,0.004001


## Add lag features for uber traveltimes

In [15]:
segment_ttime_daily = u.add_lag_features_uber(segment_ttime_daily)

segment_ttime_daily['weekday'] = segment_ttime_daily['date'].astype('datetime64[ns]').dt.weekday

segment_ttime_daily['mean_average_ttime'] = segment_ttime_daily.groupby(['segment_id', 'weekday'])['average_ttime'].transform('mean')
segment_ttime_daily['std_average_ttime'] = segment_ttime_daily.groupby(['segment_id', 'weekday'])['average_ttime'].transform('std')

segment_ttime_daily.drop('weekday', axis=1, inplace=True)

## Sun altitude and azimuth

In [17]:
sun_pos = pd.DataFrame({'datetime': pd.date_range('2016-01-01', '2019-03-31 23:00:00', freq="1h")})

sun_pos['sun_alt'] = sun_pos['datetime'].apply(u.get_alt)
sun_pos['sun_az']  = sun_pos['datetime'].apply(u.get_az)

train = u.add_time_features(train)
test  = u.add_time_features(test)

#Mean accident coordinates for each segment
coords = train_raw.groupby('road_segment_id').mean()[['longitude', 'latitude']]

## Merge train and test with generated features 

In [ ]:
weather['datetime']  = weather['datetime'].astype('datetime64[ns]')

def merge_dataframes(df):
    df = df.merge(coords, left_on='segment_id', right_on='road_segment_id', how='left')
    df[['longitude', 'latitude']] = df[['longitude', 'latitude']].round(4)
    df = df.merge(road_segments, on='segment_id', how='left')
    df = df.merge(weather, on='datetime', how='left')
    df = df.merge(sun_pos, on='datetime', how='left')
    df = df.merge(VDS_hourly, on=['datetime', 'vds_id'], how='left')
    df = df.merge(segment_ttime_daily, on=['segment_id','date'], how='left')
    return df

train = u.optimize(merge_dataframes(train))
test  = u.optimize(merge_dataframes(test))

## Calculating neighbor features for traveltime, speed and traffic

In [ ]:
# Note, this step requires a lot of time
for fval in tqdm(['traffic_total', 'avg_speed', 'average_ttime']):
    u.add_feature_based_on_neighbs(data=train, fval=fval, num_to_neighbors=num_to_neighbors)
    u.add_feature_based_on_neighbs(data=test, fval=fval, num_to_neighbors=num_to_neighbors)

## Several simple time and geometry features

In [ ]:
# Define day_period according to uber defenition
b = [0, 7, 10, 16, 19, 24]
l = ['AM', 'Early Morning','Midday','PM','Evening']
train['day_period'] = pd.cut(train['hour'], bins=b, labels=l, right=False, include_lowest=True)
test['day_period']  = pd.cut(train['hour'], bins=b, labels=l, right=False, include_lowest=True)

# Calculate angle between sun azimuth and road direction
train['blinding'] = abs(train['orientation'] - train['sun_az'])
test['blinding']  = abs(test['orientation'] - test['sun_az'])

# Calculate angle between wind and road directions
train['angle_wind_road'] = abs(train['orientation'] - train['wind_dir_angle'])
test['angle_wind_road']  = abs(test['orientation'] - test['wind_dir_angle'])

# Calculate distance from centroid to average accident location
train['dist_to_centroid'] = np.sqrt((train['lon_centroid'] - train['longitude']) ** 2 
                                    + (train['lat_centroid'] - train['latitude']) ** 2)

test['dist_to_centroid'] = np.sqrt((test['lon_centroid'] - test['longitude']) ** 2 
                                    + (test['lat_centroid'] - test['latitude']) ** 2)

## Add holidays

In [ ]:
train = u.add_public_holidays(train)
test  = u.add_public_holidays(test)
    
train = u.add_school_holidays(train)
test = u.add_school_holidays(test)

## Deleting cameras info before commission_date

In [ ]:
commission_dates.groupby(['Commissioning Date','cls_segment_id'])['Asset Type'].agg('value_counts')

train['vds_count'].loc[(train['segment_id'].isin(['5093ZHY','UY59I9M'])) & (train['datetime'] < '2016-02-05 07:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['EUXS47I','Q2V2552'])) & (train['datetime'] < '2016-07-05 12:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['YF1XDKK'])) & (train['datetime'] < '2016-07-05 12:00:00')] -= 2
train['camera_count'].loc[(train['segment_id'].isin(['812PNMZ'])) & (train['datetime'] < '2016-07-05 12:00:00')] -= 3
train['vds_count'].loc[(train['segment_id'].isin(['7QBQK9L','812PNMZ','F14EJLW'])) & (train['datetime'] < '2016-07-11 09:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['YF1XDKK'])) & (train['datetime'] < '2016-07-19 10:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['2M1UBKJ'])) & (train['datetime'] < '2016-12-08 08:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['X4UA382'])) & (train['datetime'] < '2016-12-08 09:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['N90YL69','OVJNYB2'])) & (train['datetime'] < '2017-02-02 07:00:00')] -= 1
train['vds_count'].loc[(train['segment_id'].isin(['RZK0UM0'])) & (train['datetime'] < '2017-02-15 11:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['N90YL69'])) & (train['datetime'] < '2017-03-20 07:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['B9V4IAJ'])) & (train['datetime'] < '2017-05-11 07:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['H9QJECU'])) & (train['datetime'] < '2017-10-24 10:00:00')] -= 1
train['camera_count'].loc[(train['segment_id'].isin(['MJ9YER2'])) & (train['datetime'] < '2017-11-04 07:00:00')] -= 1
train['vds_count'].loc[(train['segment_id'].isin(['8ITYQ2Z'])) & (train['datetime'] < '2017-11-07 07:00:00')] -= 1

# Also remove records from silent intervals
train = u.proc_silent_intervals(train)

## Adding last quarter count by sid-weekday-hour

In [ ]:
train['last_quarter'] = train['datetime'].dt.year + ((train['month'] - 1) // 3) * 0.25
test['last_quarter']  = test['datetime'].dt.year + ((test['month'] - 1) // 3) * 0.25

acc_cnt_quarter_sid_wdh = pd.DataFrame(train.groupby(['segment_id', 'last_quarter', 'weekday', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_sid_wd_h'})

acc_cnt_quarter_sid_wd = pd.DataFrame(train.groupby(['segment_id', 'last_quarter', 'weekday'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_sid_wd'})

acc_cnt_quarter_sid_h = pd.DataFrame(train.groupby(['segment_id', 'last_quarter', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_sid_h'})

acc_cnt_quarter_sid = pd.DataFrame(train.groupby(['segment_id', 'last_quarter'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_sid'})

acc_cnt_quarter_vds_wdh = pd.DataFrame(train.groupby(['vds_id', 'last_quarter', 'weekday', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_vds_wd_h'})

acc_cnt_quarter_vds_wd = pd.DataFrame(train.groupby(['vds_id', 'last_quarter', 'weekday'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_vds_wd'})

acc_cnt_quarter_vds_h = pd.DataFrame(train.groupby(['vds_id', 'last_quarter', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_vds_h'})

acc_cnt_quarter_vds = pd.DataFrame(train.groupby(['vds_id', 'last_quarter'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_quarter_vds'})


def add_last_quarter_counts(df: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(acc_cnt_quarter_sid_wdh, on=['segment_id', 'last_quarter', 'weekday', 'hour'], how='left')
    df = df.merge(acc_cnt_quarter_sid_wd, on=['segment_id', 'last_quarter', 'weekday'], how='left')
    df = df.merge(acc_cnt_quarter_sid_h, on=['segment_id', 'last_quarter', 'hour'], how='left')
    df = df.merge(acc_cnt_quarter_sid, on=['segment_id', 'last_quarter'], how='left')
    df = df.merge(acc_cnt_quarter_vds_wdh, on=['vds_id', 'last_quarter', 'weekday', 'hour'], how='left')
    df = df.merge(acc_cnt_quarter_vds_wd, on=['vds_id', 'last_quarter', 'weekday'], how='left')
    df = df.merge(acc_cnt_quarter_vds_h, on=['vds_id', 'last_quarter', 'hour'], how='left')
    df = df.merge(acc_cnt_quarter_vds, on=['vds_id', 'last_quarter'], how='left')
    return df

train['last_quarter'] -= 0.25
test['last_quarter']  -= 0.25

train = d.optimize(add_last_quarter_counts(train))
test  = d.optimize(add_last_quarter_counts(test))

## Adding last halfyear count by sid-weekday-hour

In [ ]:
train['last_halfyear'] = train['datetime'].dt.year + ((train['month'] - 1) // 6) * 0.5
test['last_halfyear']  = test['datetime'].dt.year + ((test['month'] - 1) // 6) * 0.5

acc_cnt_halfyear_sid_wdh = pd.DataFrame(train.groupby(['segment_id', 'last_halfyear', 'weekday', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_sid_wd_h'})

acc_cnt_halfyear_sid_wd = pd.DataFrame(train.groupby(['segment_id', 'last_halfyear', 'weekday'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_sid_wd'})

acc_cnt_halfyear_sid_h = pd.DataFrame(train.groupby(['segment_id', 'last_halfyear', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_sid_h'})

acc_cnt_halfyear_sid = pd.DataFrame(train.groupby(['segment_id', 'last_halfyear'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_sid'})


acc_cnt_halfyear_vds_wdh = pd.DataFrame(train.groupby(['vds_id', 'last_halfyear', 'weekday', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_vds_wd_h'})

acc_cnt_halfyear_vds_wd = pd.DataFrame(train.groupby(['vds_id', 'last_halfyear', 'weekday'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_vds_wd'})

acc_cnt_halfyear_vds_h = pd.DataFrame(train.groupby(['vds_id', 'last_halfyear', 'hour'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_vds_h'})

acc_cnt_halfyear_vds = pd.DataFrame(train.groupby(['vds_id', 'last_halfyear'])['y'].sum()) \
                        .reset_index().rename(columns={'y': 'acc_cnt_last_halfyear_vds'})


def add_last_halfyear_counts(df: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(acc_cnt_halfyear_sid_wdh, on=['segment_id', 'last_halfyear', 'weekday', 'hour'], how='left')
    df = df.merge(acc_cnt_halfyear_sid_wd, on=['segment_id', 'last_halfyear', 'weekday'], how='left')
    df = df.merge(acc_cnt_halfyear_sid_h, on=['segment_id', 'last_halfyear', 'hour'], how='left')
    df = df.merge(acc_cnt_halfyear_sid, on=['segment_id', 'last_halfyear'], how='left')
    df = df.merge(acc_cnt_halfyear_vds_wdh, on=['vds_id', 'last_halfyear', 'weekday', 'hour'], how='left')
    df = df.merge(acc_cnt_halfyear_vds_wd, on=['vds_id', 'last_halfyear', 'weekday'], how='left')
    df = df.merge(acc_cnt_halfyear_vds_h, on=['vds_id', 'last_halfyear', 'hour'], how='left')
    df = df.merge(acc_cnt_halfyear_vds, on=['vds_id', 'last_halfyear'], how='left')

    return df

train['last_halfyear'] -= 0.5
test['last_halfyear']  -= 0.5

train = d.optimize(add_last_halfyear_counts(train))
test  = d.optimize(add_last_halfyear_counts(test))

## Adding neighbor features for sids and last quarter/ halfyear

In [ ]:
# Note, this step requires a lot of time
for fval in tqdm(['acc_cnt_last_quarter_sid_wd_h', 'acc_cnt_last_quarter_sid_wd',
                  'acc_cnt_last_quarter_sid_h','acc_cnt_last_quarter_sid',
                  'acc_cnt_last_halfyear_sid_wd_h', 'acc_cnt_last_halfyear_sid_wd',
                  'acc_cnt_last_halfyear_sid_h', 'acc_cnt_last_halfyear_sid']
                ):
    u.add_feature_based_on_neighbs(data=train, fval=fval, num_to_neighbors=num_to_neighbors)
    u.add_feature_based_on_neighbs(data=test, fval=fval, num_to_neighbors=num_to_neighbors)
    
train = d.optimize(train)
test = d.optimize(test)

## After few simple steps we finally can save our datasets :)

In [ ]:
train.drop(['date', 'num', 'last_quarter', 'last_halfyear'], axis=1).to_pickle(train_pkl_path)  # ~7 GB
test.drop(['date', 'num', 'last_quarter', 'last_halfyear'], axis=1).to_pickle(test_pkl_path)    # ~1 GB